# Czyszczenie zbioru Titanic
### Wojciech Ogiegło
## Cel pracy

Celem pracy jest jak najlepsze przygotowanie zbioru danych o nazwie ,,TitanicMess'' w celu jego analizy. W skład przygotowania wchodzą takie rzeczy jak:
* usunięcie niepotrzebnych kolumn do analizy,
* usunięcie brakujących wartości poprzez zamianę na średnią lub usunięcie całych wystąpień, które posiadają te brakujące wartości,
* zaokrąglenie wartości do góry w zmiennych, w których niepotrzebny jest zapis dziesiętny np. wiek,
* połączenie wartości z kolumn w jedną dodatkową kolumnę.

## Rozwiązanie

Na samym początku należy zaimportować wszystkie potrzebne biblioteki:

In [1]:
import numpy as np
import pandas as pd
import math

Kolejnym krokiem jest wczytanie zbioru do zmiennej data:

In [2]:
data = pd.read_csv('TitanicMess.tsv', sep='\t', thousands=',')

Po wczytaniu danych należy się z nimi zapoznać:

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  892 non-null    int64  
 1   Survived     892 non-null    int64  
 2   Pclass       892 non-null    int64  
 3   Name         892 non-null    object 
 4   Sex          892 non-null    object 
 5   Age          719 non-null    float64
 6   SibSp        892 non-null    int64  
 7   Parch        892 non-null    int64  
 8   Ticket       892 non-null    object 
 9   Fare         892 non-null    object 
 10  Cabin        207 non-null    object 
 11  Embarked     890 non-null    object 
 12  ship         892 non-null    object 
dtypes: float64(1), int64(5), object(7)
memory usage: 90.7+ KB


In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,"8,05",NaN,S,Titanic


Jak widać, zbiór danych składa się z 891 instancji oraz 12 zmiennych. Zmienne przedstawiają się następująco:
* PassengerId - Identyfikator pasażera
* Survived - Informacja czy pasażer przeżył katastrofe, 0 - nie przeżył, 1 - przeżył
* Pclass - Klasa biletu pasażera, 1, 2 oraz 3 klasa
* Name - Imię i nazwisko pasażera
* Sex - Płeć pasażera
* Age - Wiek w latach
* SibSp - Liczba pasażerów w postaci rodzeństwa lub męża/żony, z którymi pasażer podróżował
* Parch - Liczba pasażerów w postaci rodziców lub dzieci, z którymi pasażer podróżował. Niektóre dzieci podróżowały z nianią - w tym przypadku zmienna przyjmuje wartość 0
* Ticket - Identyfikator biletu
* Fare - Opłata za bilet
* Cabin - Numer kabiny
* Embarked - Port, na którym wsiadał, C - Cherbourg, Q - Queenstown, S - Southampton
* ship - Nazwa statku

Możemy zauważyć, że kolumna Fare, w której powinny znaleźć się opłaty za bilet jest reprezentowana przez typ ,,object''. Jest to nieprawidłowe działanie, w związku z powyższym należy zamienić wszystkie ,,,'' na ,,.'', a następnie usunąć wszystkie znaki, które nie są liczbami i zamienić wartości w kolumnie na typu float.

In [5]:
data['Fare'] = data['Fare'].str.replace(',','.')
data['Fare'] = data['Fare'].str.replace(r'[^0-9\.]+', '')
data['Fare'] = pd.to_numeric(data['Fare'])

Kolejnym elementem , który dotyczy typu kolumn jest fakt, że kilka z nich ma nieprawidłowy typ danych. Kolumna Survived zawiera informacje czy ktoś przeżył katastrofę, powinna być typu kategorycznego, podobnie kolumna Pclass, Sex oraz Embarked. Poniżej zmieniono ich typ na ,,category''.

In [6]:
data["Survived"] = data["Survived"].astype("category")
data["Pclass"] = data["Pclass"].astype("category")
data["Sex"] = data["Sex"].astype("category")
data["Embarked"] = data["Embarked"].astype("category")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  892 non-null    int64   
 1   Survived     892 non-null    category
 2   Pclass       892 non-null    category
 3   Name         892 non-null    object  
 4   Sex          892 non-null    category
 5   Age          719 non-null    float64 
 6   SibSp        892 non-null    int64   
 7   Parch        892 non-null    int64   
 8   Ticket       892 non-null    object  
 9   Fare         892 non-null    float64 
 10  Cabin        207 non-null    object  
 11  Embarked     890 non-null    category
 12  ship         892 non-null    object  
dtypes: category(4), float64(2), int64(3), object(4)
memory usage: 66.9+ KB


Jak widać, operacja została wykonana pomyślnie - kolumny wymienione wyżej są typu ,,category'', a w kolumnie Fare mamy same liczby typu float.

Po głębszym zapoznaniu się ze zbiorem danych, można uznać, że kilka kolumn nie przyda się w przyszłej analizie danych. W przypadku naszego zbioru danych, takimi kolumnami jest identyfikator biletu (Ticket), numer kabiny (Cabin), nazwa statku (ship) oraz imię i nazwisko pasażera (Name). Rozwiązaniem jest usunięcie niepotrzebnych kolumn:

In [8]:
data = data.drop(['Ticket', 'Cabin', 'ship', 'Name'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  892 non-null    int64   
 1   Survived     892 non-null    category
 2   Pclass       892 non-null    category
 3   Sex          892 non-null    category
 4   Age          719 non-null    float64 
 5   SibSp        892 non-null    int64   
 6   Parch        892 non-null    int64   
 7   Fare         892 non-null    float64 
 8   Embarked     890 non-null    category
dtypes: category(4), float64(2), int64(3)
memory usage: 39.1 KB


Jak widać, kolumny zostały usunięte.

W kolejnym kroku czyszczenia danych można zauważyć, że w przypadku dwóch zmiennych mamy brakujące wartości. Zmienna Age posiada ich wiele, a zmienna Embarked posiada brakujące wartości w przypadku dwóch wystąpień. Rozwiązaniem tego problemu jest zamienienie lub usunięcie wierszy z brakującymi wartościami. Zastąpić można na kilka sposobów m.in maksymalną lub minimalną wartością występującą w tej kolumnie lub średnią. W związku z tym, że w przypadku zmiennej Age mamy wiele brakujących wartości, usunięcie ich może być problematyczne w kontekście przyszłej analizy danych. Biorąc powyższe pod uwagę, brakujące wartości w zmiennej Age zostały uzupełnione średnią z wszystkich wystąpień, a wystąpienia z brakującymi wartościami w kolumnie Embarked zostały usunięte.

In [9]:
data['Age'] = data['Age'].fillna(data['Age'].mean())
data = data[data['Embarked'].notna()]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 891
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  890 non-null    int64   
 1   Survived     890 non-null    category
 2   Pclass       890 non-null    category
 3   Sex          890 non-null    category
 4   Age          890 non-null    float64 
 5   SibSp        890 non-null    int64   
 6   Parch        890 non-null    int64   
 7   Fare         890 non-null    float64 
 8   Embarked     890 non-null    category
dtypes: category(4), float64(2), int64(3)
memory usage: 45.8 KB


Można zauważyć, że w kolumnie Age nie ma już brakujących wartości, a cały zbiór danych został pomniejszony o dwa wystąpienia, w których były brakujące wartości w kolumnie Embarked.

Kolejną rzeczą, którą można zauważyć przeglądając zbiór jest fakt, że w przypadku kolumny Age mamy wartości w postaci float - co oznacza, że np. wiek jest zapisany w następujący sposób: ,,20,2''. Nie ma sensu prezentować go w ten sposób, wystarczy, że będzie reprezentowany przez liczby całkowite. Podobnie mamy z kolumną Fare.

In [10]:
data['Age'] = data['Age'].apply(math.ceil)
data['Fare'] = data['Fare'].apply(math.ceil)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 891
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  890 non-null    int64   
 1   Survived     890 non-null    category
 2   Pclass       890 non-null    category
 3   Sex          890 non-null    category
 4   Age          890 non-null    int64   
 5   SibSp        890 non-null    int64   
 6   Parch        890 non-null    int64   
 7   Fare         890 non-null    int64   
 8   Embarked     890 non-null    category
dtypes: category(4), int64(5)
memory usage: 45.8 KB


Jak widać, mamy teraz tylo obiekty reprezentowane za pomocą liczb całkowitych lub danych kategorycznych.

Ostatnią rzeczą, którą uważam, że może przydać się do analizy jest zoptymalizowanie liczby osób, z którymi pasażer podróżował. Biorąc pod uwagę, że w zmiennej SibSp przechowywana jest liczba współpasażerów w postaci rodzeństwa lub męża/żony, a w zmiennej Parch liczba współpasażerów w postaci rodziców lub dzieci to można połączyć wartości tych dwóch zmiennych w jedną jako sumę współpasażerów danej osoby. 


In [11]:
familySize = pd.DataFrame(data.apply(lambda x: x.SibSp+x.Parch, axis=1), columns=["FamilySize"])
data = data.join(familySize)
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize
0,1,0,3,male,22,1,0,8,S,1
1,2,1,1,female,38,1,0,72,C,1
2,3,1,3,female,26,0,0,8,S,0
3,4,1,1,female,35,1,0,54,S,1
4,5,0,3,male,35,0,0,9,S,0


Jak widać powyżej - dodano do zbioru danych zmienną FamilySize, która zawiera sumę zawartości zmiennych SibSp oraz Parch. Zmienne te nie będą już potrzebne więc zostaną usunięte.

In [12]:
data = data.drop(['SibSp', 'Parch'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 891
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  890 non-null    int64   
 1   Survived     890 non-null    category
 2   Pclass       890 non-null    category
 3   Sex          890 non-null    category
 4   Age          890 non-null    int64   
 5   Fare         890 non-null    int64   
 6   Embarked     890 non-null    category
 7   FamilySize   890 non-null    int64   
dtypes: category(4), int64(4)
memory usage: 78.8 KB


Zapisanie zbioru danych do pliku:

In [13]:
data.to_csv('TitanicCleaned.tsv')

## Podsumowanie

Zbiór danych został przetworzony i zoptymalizowany w celu dalszej analizy. W trakcie tego procesu zostały wykonane wszystkie czynności wymienione we wprowadzeniu. Należą do nich m.in zapoznanie się ze zbiorem i usunięcie niepotrzebnych do dalszej analizy zmiennych, pozbycie się brakujących wartości w wystąpieniach, zamiana typów zmiennych na bardziej odpowiednie np. zmiana typu int zmiennych Survived czy Pclass na zmienne kategoryczne czy zaokrąglenie wartości w przypadku wieku i opłat co spowodowało zmianę typu z float na int. Dodatkowo została dodana dodatkowa zmienna o nazwie FamilySize, która zawiera liczbę współpasażerów danego podróżnego.